In [1]:
%cd DeepLabCut

[Errno 2] No such file or directory: 'DeepLabCut'
/home/donghan/DeepLabCut


In [2]:
import h5py
filename = '1034 SI_B, Aug 15, 13 7 49DeepCut_resnet50_ReachingMar12shuffle1_1000.h5'
f = h5py.File(filename, 'r')

# List all groups
print("Keys: %s" % f.keys())
a_group_key = list(f.keys())[0]

# Get the data
data = list(f[a_group_key])

Keys: <KeysViewHDF5 ['df_with_missing']>


In [4]:
import pandas as pd
 
with pd.HDFStore('1034 SI_B, Aug 15, 13 7 49DeepCut_resnet50_ReachingMar12shuffle1_1000.h5', 'r') as d:
    df = d.get(list(f.keys())[0])
    df.to_csv('1030 SI_B.csv')

In [5]:
data = pd.read_csv("1030 SI_B.csv",skiprows = 2)

#### Remove the first ten seconds, which has no mouse in the case (200 frames)

In [6]:
move_data = data.loc[200:]

In [7]:
import numpy as np 
def cal_dist(data): 
    fixedX = 50
    fixedY = 200
    x = data["x"]
    y = data["y"]
    dist = np.sqrt((x - fixedX)**2 + (y - fixedY)**2)  
    return dist  
cal_dist(move_data)  

200     344.118044
201     332.594098
202     323.352204
203     311.440833
204     298.166432
205     289.390415
206     286.222520
207     294.513882
208     298.088048
209     304.539540
210     312.963229
211     315.736029
212     322.156042
213     323.924807
214     401.153496
215     399.041802
216     380.849401
217     339.234834
218     335.810031
219     332.707384
220     341.860259
221     343.504520
222     338.364962
223     333.339765
224     330.796884
225     328.025120
226     327.719281
227     325.973455
228     326.048267
229     323.770524
           ...    
3451    123.229294
3452    135.945594
3453    136.956514
3454    145.298398
3455    148.575912
3456    154.560320
3457    159.360432
3458    166.001592
3459    173.947869
3460    179.933533
3461    180.228322
3462    176.634929
3463    180.111036
3464    182.164312
3465    185.772060
3466    191.275780
3467    198.836854
3468    204.424935
3469    211.783919
3470    220.797657
3471    225.549971
3472    223.

In [8]:
# Reference
# https://stackoverflow.com/questions/42258637/how-to-know-the-angle-between-two-point
from math import degrees, atan2
def head_dir(data):
    p1 = pd.concat([data["x"], data["y"]],axis=1)
    p2 = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y'])
    xDiff = p2.x - p1.x
    yDiff = p2.y - p1.y
    degree = []
    for i in range(0,len(xDiff)):
        degree.append(degrees(atan2(tuple(yDiff)[i], tuple(xDiff)[i])))
    direction = [x-x+1 if x >= 90 or x <= -90 else x-x for x in degree]
    return (direction, degree)

head_dir(move_data)
# Whether the mouse's direction towards bullying mouse or not, 1: yes; 0: no
# The degree of head direction, clockwise.

([1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,